### 4.2 BPR排序算法

#### 一. BPR的概念
1. 推荐系统, 本质上是针对一系列物品的个性化排序问题.BPR使用贝叶斯思想来优化排序效果
2. BPR关注的是隐式反馈行为
3. SVD的矩阵分解模型(MF-matrix factorization)和KNN模型使用BPR-OPT优化函数后, 可以达到更好的效果
4. BPR用于协同过滤模型时, 会对每个用户产生各自的排序效果, 而非产生一个全局的统一排序

#### 二. 符号表示
1. $U$ : 用户集合. $I$ : 所有物品集合
3. $>_U$ : 所有物品的偏序关系,维度是$I^2$. 且$>_U$满足如下3个性质 : 
   1. 全体性 : $\forall i,j\in I$, $i\neq j\Longrightarrow i{ > }_{ u }j\vee j{ > }_{ u }i$  
   2. 反对称性 : $\forall i,j\in I$, $\left( i{ > }_{ u }j \right) \wedge \left( j{ > }_{ u }i \right) \Longrightarrow i=j$ 
   3. 传递性 : $\forall i,j,k\in I$, $\left( i{ > }_{ u }j \right) \wedge \left( j{ > }_{ u }k \right) \Longrightarrow \left( i{ > }_{ u }k \right) $
3. $S$ : 隐式反馈矩阵,维度是$U*I$. 由0和1组成  
 <img src='img/Sbpr.png' height='40%' width='40%'>
4. ${ I }_{ u }^{ + }=\left\{ i\in I:\left( u,i \right) \in S \right\} $ : 对于用户u, 做出反馈的物品集合    
 ${ U }_{ i }^{ + }=\left\{ u\in U:\left( u,i \right) \in S \right\} $ : 对于物品i, 做出反馈的用户集合
 
#### 三. BPR的训练集构造  
1. BPR的训练集是一个3元组(u,i,j). 其中i是用户做出反馈的1个物品, j是没有作出反馈的1个物品. 于是我们认为对于用户u, 作出反馈的物品i的排序应该在未作出反馈的物品j之前. 训练集表示为 : $${ D }_{ S }=\left\{ (u,i,j)|i\in { I }_{ u }^{ + }\wedge j\in I\setminus { I }_{ u }^{ + } \right\} $$即$\left( u,i,j \right) \in { D }_{ S }$表示用户u相比j更喜欢i.  
2. 现在, 我们可以从观测矩阵$R^{U*I}$,得到每个用户自己的偏序关系矩阵$R^{I*I}$,因此所有用户的偏序关系矩阵维度是$R^{U*I*I}$. 如下图, '+'表示更喜欢$i$, '-'表示更喜欢$j$
  <img src='img/bpr2.png' height='40%' width='40%'>  
  注意, 我们的训练集是由做出反馈的i和未作出反馈的j组成, 而测试集输入的物品都是未反馈的. 这说明训练集和测试集是割裂的

#### 四. BPR-OPT : BPR优化准则
1. 贝叶斯寻找最优排序的方法是, 最大化参数的后验概率(偏序关系下的参数概率)$p\left( \theta |{ > }_{ u } \right) $, 且根据贝叶斯公式有 : $$p\left( \theta |{ > }_{ u } \right) \propto p\left( { > }_{ u }|\theta  \right) p\left( \theta  \right) $$在所有用户集上的似然概率为 : $$\prod _{ u\in U }^{  }{ p\left( { > }_{ u }|\theta  \right)  } =\prod _{ \left( u,i,j \right) \in U*I*I }^{  }{ \left\{ { p\left( i{ > }_{ u }j|\theta  \right)  }^{ \delta \left( \left( u,i,j \right) \in { D }_{ S } \right)  }*{ \left( 1-p\left( i{ > }_{ u }j|\theta  \right)  \right)  }^{ \delta \left( \left( u,i,j \right) \notin { D }_{ S } \right)  } \right\}  } $$  
  1. 其中, $\delta \left( b \right) $是指数函数, 有$\delta \left( b \right) =\begin{cases} 1,\quad b=true \\ 0,\quad else \end{cases}$
  2. $p\left( i{ > }_{ u }j|\theta  \right) =sigmoid\left( { \hat { x }  }_{ uij }\left( \theta  \right)  \right) =\frac { 1 }{ 1+{ e }^{ -{ \hat { x }  }_{ uij }\left( \theta  \right)  } } $.   
    这里的${ \hat { x }  }_{ uij }\left( \theta  \right) $是任意实值函数, 表示用户,物品i,物品j3这的关系. 在MF和KNN中将有不同表示方法.  
  3. 为了方便计算, 假设先验概率服从正态分布, 即$\theta =N\left( 0,{ \Sigma  }_{ \theta  } \right) $. 为了简便计算, 我们设${ \Sigma  }_{ \theta  }={ \lambda  }_{ \theta  }I$  
2. 综合以上后验概率计算方法, 得出BPR-OPT优化使用的最大化对数似然为$$BPR-OPT=\sum _{ \left( u,i,j \right) \in U*I*I }^{  }{ \delta \left( \left( u,i,j \right) \in { D }_{ S } \right) \ln { p\left( i{ > }_{ u }j|\theta  \right)  } } +\delta \left( \left( u,i,j \right) \notin { D }_{ S } \right) \ln { \left( 1-p\left( i{ > }_{ u }j|\theta  \right)  \right)+\ln { p\left( \theta  \right)  }   } $$因为接下来, 我们的样本集都是从$D_S$中得出,所以指示函数${ \delta \left( \left( u,i,j \right) \in { D }_{ S } \right)  }=1$,${ \delta \left( \left( u,i,j \right) \notin { D }_{ S } \right)  }=0$, 因此有 :   
$$BPR-OPT=\sum _{ \left( u,i,j \right) \in { D }_{ S } }^{  }{ \ln { p\left( i{ > }_{ u }j|\theta  \right)  } +\ln { p\left( \theta  \right)  }  } \\  \quad \quad \quad \quad =\sum _{ \left( u,i,j \right) \in { D }_{ S } }^{  }{ \ln { p\left( i{ > }_{ u }j|\theta  \right)  }  } -{ \lambda  }_{ \theta  }{ \left\| \theta  \right\|  }^{ 2 }$$  
3. 以上, 我们得出了'贝叶斯个性化排序的优化函数'和'训练集构造方法', 再加上'BPR-learn'计算方法就得出了全部BPR的训练方法.  
 因此, BPR是一种最大化后验概率likely-hood的解决办法. 具体的用户u,物品i,物品j之间的关系需要靠MF或KNN算法给出

#### 五. BPR-Learning
1. `BPR-Learning`是利用随机梯度下降最小化`BPR-OPT`, 从而更新参数$\theta $ 
<img src='img/bprlearn2.png' height='85%' width='85%'>

2. ${ \hat { x }  }_{ uij }$到底是什么  
在`BPR-OPT`的后验概率中, 就包含${ \hat { x }  }_{ uij }$, 梯度更新中也包含${ \hat { x }  }_{ uij }$, 我们知道这代表了用户u,物品i,物品j三者之间的关系. 现在, 我们假设$${ \hat { x }  }_{ uij }={ \hat { x }  }_{ ui }-{ \hat { x }  }_{ uj }$$其中, ${ \hat { x }  }_{ ui }$和${ \hat { x }  }_{ uj }$可以来自协同过滤矩阵分解(MF-matrix factorization)或者KNN   
  1. 使用MF计算时  
   ${ \hat { x }  }_{ ui }$为用户向量和物品向量的点积$${ \hat { x }  }_{ ui }=\left< { w }_{ u },{ h }_{ i } \right> =\sum _{ f=1 }^{ k }{ { w }_{ uf }{ h }_{ if } } $$所以,针对3各参数的梯度下降为
      <img src='img/bpr31.png' weight='74%' width='74%'>
  2. 使用KNN计算时  
   ${ \hat { x }  }_{ ui }$的值, 依赖于物品i和所有用户已消费过的物品集合的相似度, 所以$${ \hat { x }  }_{ ui }=\sum _{ l\in { I }_{ u }^{ + }\wedge l\neq i }^{  }{ { c }_{ il } } \quad ({ c }_{ il }为物品i和物品l之间的相似度, 可使用cos夹角表示)$$因此, 此时${ \hat { x }  }_{ ui }=参数c$, 就有梯度为
   <img src='img/bpr42.png' width='78%' height='78%'>

### 4.3 movieLens 100K基于BPR推荐

#### 一. 数据预处理(data/u.data)
1. 数据文件中, 每行有4列, 前3列分别为用户id,电影id,和用户给电影的评分
2. 因为我们要BRP, 所以要将显式评分转换成隐式的用户看过哪些电影

### 二. 使用tensorflow训练
[http://www.cnblogs.com/pinard/p/9163481.html](http://www.cnblogs.com/pinard/p/9163481.html)


In [12]:
open('data/u.data')

<_io.TextIOWrapper name='data/u.data' mode='r' encoding='UTF-8'>